## Importing Modules


In [ ]:
!pip install torchmetrics

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 KB 22.8 MB/s eta 0:00:00


In [ ]:
!pip install pickle5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 KB 9.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp39-cp39-linux_x86_64.whl size=255890 sha256=98aa7405a59a62c55f1cdd7f324ff3580416b63f961a2fd6724ad201ab04cb19
  Stored in directory: /root/.cache/pip/wheels/f2/7a/49/9bef8878949914ecb90c08fc5bf30a05e17f475fe7e08b63a8
Successfully built pickle5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import pickle5 as pickle
import pandas as pd
import os
import torch.optim as optim
import torch
from tqdm import tqdm
from torchmetrics.functional.classification import multilabel_accuracy
import numpy as np
from sklearn.metrics import classification_report

from sklearn.model_selection import StratifiedShuffleSplit

from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

import warnings
warnings.filterwarnings("ignore")
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

### Support

In [ ]:
def load_pkl(filename):
    filename= filename
    data= None
    with open(filename, "rb") as handle:
        data= pickle.load(handle)
        handle.close()
    return data


def checkpoint(model, filename):
    torch.save(model.state_dict(), filename)
    
def resume(filename):
    model.load_state_dict(torch.load(filename))

### DataSet Loader


In [ ]:
from ast import literal_eval
class customDataset(torch.utils.data.Dataset):
    def __init__(self, data_file ,video_file_name, uservideo_file_name,audio_file_name, useraudio_file_name):
        
        self.triplet = data_file
        self.UserVideo_representation = load_pkl(uservideo_file_name)
        self.video_representation = load_pkl(video_file_name)       
        # self.UserAudio_representation = load_pkl(useraudio_file_name)
        # self.audio_representation = load_pkl(audio_file_name)   
        
    def __len__(self):
        return len(self.triplet)
    
    def __getitem__(self, idx):
        user_id , movie_id , pref_score , rating = self.triplet[idx]
        
        #Video Embedding 
        UserVideo_embed = torch.FloatTensor(self.UserVideo_representation[user_id])
        video_embed = pref_score * 100 * torch.FloatTensor(self.video_representation[movie_id])
        
        #Audio Embedding 
        UserAudio_embed = torch.FloatTensor(self.UserVideo_representation[user_id])
        audio_embed = pref_score * 100 * torch.FloatTensor(self.video_representation[movie_id])
        
        #Label
        y_label = torch.FloatTensor(self.onehot(rating[0]))
        
        return movie_id, UserVideo_embed, video_embed, UserAudio_embed, audio_embed, y_label
    
    def onehot(self,rate):
        if rate == '-1':
            return([1.,0.,0.])           
               
        elif rate == '1':
            return([0.,0.,1.])           
               
        elif rate == '0':
            return([0.,1.,0.])


### Report

In [ ]:
def cal_accuracy(y_test, y_pred):
    with open("classification_report_audio_video_all.txt", "a") as h:
        print("\n")  
        # print("Confusion Matrix:\n ",confusion_matrix(y_test, y_pred))
        print("Confusion Matrix:\n ",confusion_matrix(y_test, y_pred), file=h) 

        print ("Accuracy :\n ",accuracy_score(y_test,y_pred)*100, file=h) 

        print("Report : \n", classification_report(y_test, y_pred), file=h)
        # print("Report : \n", classification_report(y_test, y_pred))
        precision = precision_score(y_test, y_pred, average = 'micro')
        print('Precision: %f' % precision, file=h)
        # recall: tp / (tp + fn)
        recall = recall_score(y_test, y_pred, average = 'micro')
        print('Recall: %f' % recall, file=h)
        # f1: 2 tp / (2 tp + fp + fn)
        f1 = f1_score(y_test, y_pred, average = 'micro')
        print('F1 score: %f' % f1, file=h)


### Loading Files

In [ ]:
#Movie Embeddings 

audio_file_name = "/content/drive/MyDrive/Recommendation_Journal/movie_embedding/Video Embeddings TimesFormer Based/FlickScore_Movie_Audio_User_embedding/Audio_Embedding/w2v_genre/genre_based_movie_audio_embedding.pkl"
video_file_name = "/content/drive/MyDrive/Recommendation_Journal/movie_embedding/Video Embeddings TimesFormer Based/FlickScore_Movie_Audio_User_embedding/Movie_Embedding/frames_in_equal_interval_video_embeddings/frames_in_equal_interval_video_embeddings_dict.pkl"

#User Embeddings
useraudio_file_name = "/content/drive/MyDrive/Recommendation_Journal/movie_embedding/Video Embeddings TimesFormer Based/FlickScore_Movie_Audio_User_embedding/Audio_Embedding/w2v_genre/user_emb_all_Score.pkl"
uservideo_file_name = "/content/drive/MyDrive/Recommendation_Journal/movie_embedding/Video Embeddings TimesFormer Based/FlickScore_Movie_Audio_User_embedding/Movie_Embedding/frames_in_equal_interval_video_embeddings/user_emb_all_Score.pkl"

#Data Triplet 
data_file_final = load_pkl("/content/drive/MyDrive/scene_detect/data_file_final.pkl")
print(len(data_file_final))

21479


In [ ]:
# #Movie Embeddings 

# # audio_file_name = "/home/prabir_prj22/Pulkit/User_Modality_Model/Dataset/MovieEmbeddings/genre_based_movie_audio_embedding.obj"
# video_file_name = load_pkl("/content/drive/MyDrive/Recommendation_Journal/movie_embedding/Video Embeddings TimesFormer Based/FlickScore_Movie_Audio_User_embedding/Movie_Embedding/frames_in_equal_interval_video_embeddings/frames_in_equal_interval_video_embeddings_dict.pkl")

# #User Embeddings
# # useraudio_file_name = "/home/prabir_prj22/Pulkit/User_Modality_Model/Dataset/UserEmbeddings/pref_score_based_user_audio_embedding.obj"
# uservideo_file_name = load_pkl("/content/drive/MyDrive/Recommendation_Journal/movie_embedding/Video Embeddings TimesFormer Based/FlickScore_Movie_Audio_User_embedding/Movie_Embedding/frames_in_equal_interval_video_embeddings/user_emb_like_Score.pkl")

# #Data Triplet 
# data_file_final = load_pkl("/content/drive/MyDrive/scene_detect/data_file_final.pkl")
# print(len(data_file_final))

## Model 

In [ ]:
class Recom_model(torch.nn.Module):
    def __init__(self):
        super(Recom_model, self).__init__()

        self.movie_block = torch.nn.Sequential(
            torch.nn.Linear(in_features=512, out_features=256),
            torch.nn.Tanh(),
            torch.nn.Linear(in_features=256, out_features=128),
            torch.nn.Tanh(),
            torch.nn.Linear(in_features=128, out_features=64),
            torch.nn.Tanh())
        
        self.user_block = torch.nn.Sequential(
            torch.nn.Linear(in_features=512, out_features=256),
            torch.nn.Tanh(),
            torch.nn.Linear(in_features=256, out_features=128),
            torch.nn.Tanh(),
            torch.nn.Linear(in_features=128, out_features=64),
            torch.nn.Tanh())
        
        self.classification_block = torch.nn.Sequential(
            torch.nn.Linear(in_features=256, out_features=128),
            torch.nn.Tanh(),
            torch.nn.Linear(in_features=128, out_features=64),
            torch.nn.Tanh(),
            torch.nn.Linear(in_features=64, out_features=3),
        )

    def forward(self, movie_audio,movie_video,user_audio,user_video):
        
        movie_audio_embed = self.movie_block(movie_audio)
        movie_video_embed = self.movie_block(movie_video)
        # print("feature.shape", movie_embed.shape)
        
        user_audio_embed = self.user_block(user_audio)
        user_video_embed = self.user_block(user_video)

        total_user = torch.cat((user_audio_embed,user_video_embed), 1)
        total_movie = torch.cat(( movie_audio_embed , movie_video_embed ), 1)

        user_movie = torch.cat((total_user, total_movie), 1)
        # print("feature.shape", user_movie.shape)
        pred = self.classification_block(user_movie)
        return pred

### Train Function

In [ ]:
def train(model, dataloader, optimizer, criterion, train_data, device):
    print('Training')
    model.train()
    counter = 0
    train_running_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_data)/dataloader.batch_size)):
        counter += 1
        movie_id, x_UserVideo, x_video, x_UserAudio, x_audio, target = data
        
        x_UserVideo =  x_UserVideo.to(device)
        x_video = x_video.to(device)
        x_UserAudio =  x_UserAudio.to(device)
        x_audio = x_audio.to(device)
        target = target.to(device)
        # print(target)

        
        optimizer.zero_grad()
        outputs = model(x_audio, x_video, x_UserAudio, x_UserVideo)  
        # loss = torch.nn.functional.cross_entropy(outputs, target)       
        loss = criterion(outputs, target)
        train_running_loss += loss.item()
        
        # backpropagation
        loss.backward()
        
        # update optimizer parameters
        optimizer.step()

    train_loss = train_running_loss / counter
    return train_loss

### Test-Function

In [ ]:
def validate(model, dataloader, criterion, val_data, device):
    print('Validating')
    model.eval()
    counter = 0
    val_running_loss = 0.0
    running_accuracy = 0.0
    
    with torch.no_grad():
        for i, data in tqdm(enumerate(dataloader), total=int(len(val_data)/dataloader.batch_size)):
            counter += 1
            movie_id, x_UserVideo, x_video, x_UserAudio, x_audio, target = data
        
            x_UserVideo =  x_UserVideo.to(device)
            x_video = x_video.to(device)
            x_UserAudio =  x_UserAudio.to(device)
            x_audio = x_audio.to(device)
            target = target.to(device)
            # print(target)

            # outputs = model(x_UserVideo, x_video)     
            outputs = model(x_audio, x_video, x_UserAudio, x_UserVideo)                     
            loss = criterion(outputs, target)       
            val_running_loss += loss.item()

            #Calculating Accuracy                               
            accuracy = (torch.argmax(outputs, 1) == torch.argmax(target, 1)).float().mean()

        val_loss = val_running_loss / counter        
        return val_loss , accuracy , target , outputs

# Train Test Split

In [ ]:
#Train - Test Split
X = []
Y = []
for sample in data_file_final:
    X.append(sample)
    Y.append(int(sample[3][0]))  

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for i, (train_index, test_index) in enumerate(sss.split(X, Y)):
    train_data_X = []
    test_data = []
    
    for trn_indx in train_index:
        train_data_X.append(X[trn_indx])

    for tst_indx in test_index:
        test_data.append(X[tst_indx]) 

train_set = customDataset(train_data_X, video_file_name, uservideo_file_name, audio_file_name, useraudio_file_name)

In [ ]:
a = iter(train_set)

# Five Fold Training

In [ ]:

# learning parameters
from torch.optim.lr_scheduler import StepLR
early_stop_thresh = 15
best_loss = 1000
epochs = 150

print("Model Transfered")


#Train - Test Split
X = []
Y = []
for sample in data_file_final:
    X.append(sample)
    Y.append(int(sample[3][0]))    

import collections
# using Counter to find frequency of elements
frequency = collections.Counter(Y)
print(dict(frequency))


sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

for i, (train_index, test_index) in enumerate(sss.split(X, Y)):
    train_data_X = []
    test_data = []
    
    for trn_indx in train_index:
        train_data_X.append(X[trn_indx])

    for tst_indx in test_index:
        test_data.append(X[tst_indx]) 


#Train - Val Split
X = []
Y = []
for sample in train_data_X:
    X.append(sample)
    Y.append(int(sample[3][0]))

sss1 = StratifiedShuffleSplit(n_splits=5, test_size=0.1, random_state=0)


#Running for 5 Splits 
for i, (train_index, val_index) in enumerate(sss1.split(X, Y)):
    best_epoch = -1
    best_loss = 1000
    lr = 0.00001
    # fold = 'fold_'+ str(i)
    train_data = []
    val_data = []

    for trn_indx in train_index:
        train_data.append(X[trn_indx])

    for val_indx in val_index:
        val_data.append(X[val_indx]) 

    #Definig Loaders
    train_set = customDataset(train_data, video_file_name, uservideo_file_name, audio_file_name, useraudio_file_name)
    val_set = customDataset(val_data, video_file_name, uservideo_file_name, audio_file_name, useraudio_file_name)
    test_set = customDataset(test_data, video_file_name, uservideo_file_name, audio_file_name, useraudio_file_name)

    print("Data in Train set:" , len(train_set))
    print("Data in Val set:" , len(val_set))
    print("Data in Test set:" , len(test_set))
    
    train_loader = torch.utils.data.DataLoader(dataset = train_set , batch_size=32, shuffle=True)
    val_loader = torch.utils.data.DataLoader(dataset = val_set , batch_size=len(val_set))   
    
                                              
    #Trainig Loop
    model = Recom_model()
    model = model.to(device)
    optimizer = optim.RMSprop(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, 
                   step_size = 50, # Period of learning rate decay
                   gamma = 0.1) # Multiplicative factor of learning rate decay
    criterion = nn.CrossEntropyLoss()
    #Contains train_loss and valid_loss for the complete epochs
    train_loss = []
    valid_loss = []
    valid_acc = []
    min_valid_loss = 1000.0

    for epoch in range(epochs):
        scheduler.step()        
        print(f"Epoch {epoch+1} of {epochs}" , "Learnig Rate: " ,scheduler.get_last_lr())

        #CALLING TRAINING FUNCTION
        train_epoch_loss = train(
            model, train_loader, optimizer, criterion, train_set, device
        )

        #CALLING VALIDATION FUNCTION
        valid_epoch_loss , accuracy_score_epoch , target , outputs = validate(
            model, val_loader, criterion, val_set, device
        )

        train_loss.append(train_epoch_loss)
        valid_loss.append(valid_epoch_loss)
        valid_acc.append(accuracy_score_epoch.item())
        print(f"Train Loss: {train_epoch_loss:.4f}")
        print(f'Val Loss: {valid_epoch_loss:.4f}')
        print(f'Val Acc: {accuracy_score_epoch:.4f}')
                
        if valid_epoch_loss < best_loss:
            best_loss = valid_epoch_loss
            best_epoch = epoch
            checkpoint(model, "best_model_ALL.pth")
        elif epoch - best_epoch > early_stop_thresh:
            print("Early stopped training at epoch %d" % epoch)
            break  # terminate the training loop
    
    #Testing The model
    resume("best_model_ALL.pth")
    model.to(device)
    test_loader = torch.utils.data.DataLoader(dataset = test_set , batch_size=len(test_set))
    print(len(test_set))

    test_epoch_loss , Testaccuracy_score_epoch, target , outputs = validate(
            model, test_loader, criterion, test_set, device
        )
    y_true = target.cpu().data.numpy().argmax(axis=-1)
    y_pred = outputs.cpu().data.numpy().argmax(axis=-1)
    cal_accuracy(y_true, y_pred)
    print(f'Test Loss: {test_epoch_loss:.4f}')
    print(f'Test Acc: {Testaccuracy_score_epoch:.4f}')


Model Transfered
{1: 6981, 0: 7873, -1: 6625}
Data in Train set: 15464
Data in Val set: 1719
Data in Test set: 4296
Epoch 1 of 150 Learnig Rate:  [1e-05]
Training


484it [00:06, 75.54it/s]


Validating


100%|██████████| 1/1 [00:00<00:00,  1.35it/s]


Train Loss: 1.0537
Val Loss: 1.0142
Val Acc: 0.5073
Epoch 2 of 150 Learnig Rate:  [1e-05]
Training


...

Validating


100%|██████████| 1/1 [00:00<00:00,  2.49it/s]


Train Loss: 0.6428
Val Loss: 0.7201
Val Acc: 0.6998
Early stopped training at epoch 115
4296
Validating


100%|██████████| 1/1 [00:00<00:00,  1.01it/s]



Test Loss: 0.7064
Test Acc: 0.7004
